In [1]:
# STEP 1: LOADING THE DATASET AND UNDERSTANDING THE DATASET AND COLUMN SELECTION
import pandas as pd

# Load the dataset to get column names
df = pd.read_csv('SupplyChainDataset.csv', encoding='ISO-8859-1')
print(df.columns.tolist())
print(df.info())

['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Delivery Status', 'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City', 'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Customer Password', 'Customer Segment', 'Customer State', 'Customer Street', 'Customer Zipcode', 'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market', 'Order City', 'Order Country', 'Order Customer Id', 'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status', 'Order Zipcode', 'Product Card Id', 'Product Category Id', 'Product Description', 'Product Image', 'Product Name', 'Product Price', 'Product Status', 'shipping date (DateOrde

In [2]:
import pandas as pd

# Load the dataset
# Based on the previous output, it seems the entire row might be quoted or there's a delimiter issue.

# Let's try reading it normally first and check the columns again.

df = pd.read_csv('Supplychain project.csv', encoding='ISO-8859-1')

# Check if the dataframe has only one column (which would mean it didn't parse correctly)
if len(df.columns) == 1:
    # It might be that the file is actually a CSV inside a single column or has weird quoting.
    # Let's try to reload with specific parameters if needed.
    # Re-reading using the first column as header if it's just one big string
    import csv
    df = pd.read_csv('supplychain project.csv', encoding='ISO-8859-1', quotechar='"', skipinitialspace=True)

print("Columns:", df.columns.tolist())
print("Shape:", df.shape)
print("Head:\n", df.head())
print("Data Types:\n", df.dtypes)

Columns: ['ï»¿"Type', 'Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Delivery Status', 'Late_delivery_risk', 'Category Id', 'Category Name', 'Cust_City', 'Cust_Country', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Customer Password', 'Customer Segment', 'Customer State', 'Customer Street', 'Customer Zipcode', 'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market', 'Order City', 'Order Country', 'Order Customer Id', 'Order_Date', 'Order_id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Qnty', 'Sales', 'Order Item Total', 'Profit', 'Order Region', 'Order State', 'Order Status', 'Order Zipcode', 'Product Card Id', 'Product_Category', 'Product Description', 'Product Image', 'Product Name', 'Cost', 'Product Status', 'Shipping_Date', 'Shipping Mode"']
Shape: (180519, 53)
Head:
              

In [3]:
with open('supplychain project.csv', 'r', encoding='ISO-8859-1') as f:
    for i in range(5):
        print(repr(f.readline()))

'ï»¿"Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Cust_City,Cust_Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,Order_Date,Order_id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Qnty,Sales,Order Item Total,Profit,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product_Category,Product Description,Product Image,Product Name,Cost,Product Status,Shipping_Date,Shipping Mode"\n'
'"DEBIT,3,4,91.25,314.6400146,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.2514534,-66.03705597,P

In [1]:
# STEP 2: DATA CLEANING 

import pandas as pd
import io

# Step 0: Correctly read the strangely formatted CSV
with open('supplychain project.csv', 'r', encoding='ISO-8859-1') as f:
    cleaned_lines = [line.strip().strip('"') for line in f]

df = pd.read_csv(io.StringIO('\n'.join(cleaned_lines)))

# Initial check
print("Shape before cleaning:", df.shape)

# Define key columns for supply chain optimization
key_columns = ['Order_id', 'Order_Date', 'Shipping_Date', 'Product_Category', 
               'Cust_City', 'Cust_Country', 'Qnty', 'Sales', 'Profit', 'Cost']

# 1. Handling missing values in key columns
# Check missing values
print("Missing values in key columns:\n", df[key_columns].isnull().sum())
# Strategy: drop rows where key columns are missing
df = df.dropna(subset=key_columns)

# 2. Remove duplicate rows
df = df.drop_duplicates()

# 3. Standardize cost and profit units to numeric
df['Cost'] = pd.to_numeric(df['Cost'], errors='coerce')
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')
# Also ensure Sales and Qnty are numeric
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')
df['Qnty'] = pd.to_numeric(df['Qnty'], errors='coerce')

# Drop any rows where numeric conversion failed for essential metrics
df = df.dropna(subset=['Cost', 'Profit', 'Sales', 'Qnty'])

# 4. Creating a new column for shipping_delay = Shipping_Date - Order_Date
# Convert date columns to datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Shipping_Date'] = pd.to_datetime(df['Shipping_Date'])

# Calculate delay in days
df['shipping_delay'] = (df['Shipping_Date'] - df['Order_Date']).dt.days

# Final check
print("Shape after cleaning:", df.shape)
print("Cleaned Data Head:\n", df[['Order_Date', 'Shipping_Date', 'shipping_delay', 'Profit', 'Cost']].head())

# Save the cleaned dataset
df.to_csv('cleaned_supplychain_dataset.csv', index=False)

Shape before cleaning: (180519, 53)
Missing values in key columns:
 Order_id            0
Order_Date          0
Shipping_Date       0
Product_Category    0
Cust_City           0
Cust_Country        0
Qnty                0
Sales               0
Profit              0
Cost                0
dtype: int64
Shape after cleaning: (180519, 54)
Cleaned Data Head:
            Order_Date       Shipping_Date  shipping_delay      Profit    Cost
0 2018-01-31 22:56:00 2018-02-03 22:56:00               3   91.250000  327.75
1 2018-01-13 12:27:00 2018-01-18 12:27:00               5 -249.089996  327.75
2 2018-01-13 12:06:00 2018-01-17 12:06:00               4 -247.779999  327.75
3 2018-01-13 11:45:00 2018-01-16 11:45:00               3   22.860001  327.75
4 2018-01-13 11:24:00 2018-01-15 11:24:00               2  134.210007  327.75
